# Preliminary experiment

In this setup there is an ensemble (a) which gets stimulation inputs in 6 dimensions: x, y and z axes and the Aircraft principal axes roll, pitch and yaw. The ensamble a integrates the inputs on each axis in order to calculate the correction signal which bring the system to the target state (in our first case - the origin (0, 0, 0, 0, 0, 0))

In [13]:
# package installation
!pip install nengo
!pip install nengo-gui
!pip install numpy
!pip install matplotlib

In [14]:
%matplotlib inline
import matplotlib.pyplot as plt
import nengo
from nengo.processes import Piecewise

In [15]:
model = nengo.Network()
with model:
    
    # create the stimuli. stimuli are separated for simplicity
    stim_x = nengo.Node(0)
    stim_y = nengo.Node(0)
    stim_z = nengo.Node(0)
    stim_roll = nengo.Node(0)
    stim_pitch = nengo.Node(0)
    stim_yaw = nengo.Node(0)
    
    # create ensamble a
    a = nengo.Ensemble(n_neurons=50, dimensions=1)
    
    # connect the stimuli to ensamble a
    nengo.Connection(stim_x, a)
    nengo.Connection(stim_y, a)
    nengo.Connection(stim_z, a)
    nengo.Connection(stim_roll, a)
    nengo.Connection(stim_pitch, a)
    nengo.Connection(stim_yaw, a)
    
    # create ensamble for the integration
    integration = nengo.Ensemble(n_neurons=50, dimensions=1)
    
    # create ensamble which represent the target state
    target = nengo.Ensemble(n_neurons=50, dimensions=1)
    
    # create ensamble which represent the correction signal which brings the
    # ensamble a to the target state
    correction_signal = nengo.Ensemble(n_neurons=50, dimensions=1)
    
    # connect ensamble a to the integration ensamble
    nengo.Connection(a, integration)
    
    # connect the target ensamble to the integration ensamble
    nengo.Connection(target, integration)
    
    # connect the integration ensamble to the correction signal ensamble
    nengo.Connection(integration, correction_signal)


In [16]:
# import nengo

# model = nengo.Network()
# with model:

#     x = nengo.Ensemble(n_neurons=400, dimensions=3)

#     synapse = 0.1

#     def oscillator(x):
#         r = 1
#         s = 10 * x[2]
#         return [synapse * -x[1] * s + x[0] * (r - x[0]**2 - x[1]**2) + x[0],
#                 synapse *  x[0] * s + x[1] * (r - x[0]**2 - x[1]**2) + x[1]]

#     nengo.Connection(x, x[:2], synapse=synapse, function=oscillator)

#     stim_speed = nengo.Node(0)
#     speed = nengo.Ensemble(n_neurons=50, dimensions=1)
#     nengo.Connection(stim_speed, speed)
#     nengo.Connection(speed, x[2])


In [17]:
with nengo.Simulator(model) as sim:
    sim.run(5)

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

AttributeError: module 'scipy.linalg' has no attribute 'cho_factor'

In [18]:
# plt.figure()
# plt.plot(sim.trange(), sim.data[oscillator_probe])
# plt.xlabel('Time (s)')
# plt.legend(['$x_0$', '$x_1$', r'$\omega$']);

In [19]:
with nengo.Simulator(model) as sim:
    sim.run(5)

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

AttributeError: module 'scipy.linalg' has no attribute 'cho_factor'

In [20]:
import nengo
net = nengo.Network()
sim = nengo.Simulator(net)
sim.run(5.0)


HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.

HtmlProgressBar cannot be displayed. Please use the TerminalProgressBar. It can be enabled with `nengo.rc.set('progress', 'progress_bar', 'nengo.utils.progress.TerminalProgressBar')`.